<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

import credit_scoring_kernel_module as cm

In [2]:
pd.set_option('display.max_columns', 250) # показывать больше колонок
pd.set_option('display.max_rows', 250) # показывать больше строк

In [3]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
Numpy        : 1.18.5


In [4]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [5]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [6]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [7]:
VERSION    = 15
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TRAIN_NEW = '../input/all-msk-auto-nov-2020/'
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# CATBOOST
ITERATIONS = 5000
LR         = 0.1

# Data

In [8]:
!ls '../input'

all-msk-auto-nov-2020		       sf-dst-car-price-prediction
parsing-all-moscow-auto-ru-09-09-2020


In [9]:
# train = pd.read_csv(DIR_TRAIN +'all_auto_ru_09_09_2020.csv') # датасет для обучения модели

train = pd.read_csv(DIR_TRAIN_NEW +'train.csv')

test = pd.read_csv(DIR_TEST +'test.csv')

sample_submission = pd.read_csv(DIR_TEST +'sample_submission.csv')

In [10]:
train.head(5)

,Unnamed: 0,index,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,location,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,0,0,внедорожник 5 дв.,VOLVO,https://auto.ru/cars/used/sale/volvo/xc90/1101...,синий,{'id': '0'},Продажа а/м осуществляется ОФИЦИАЛЬНЫМ ДИЛЕРОМ...,2.0 LTR,235 N12,Москва,"{'esp': True, 'airbag-driver': True, 'aux': Tr...",дизель,//avatars.mds.yandex.net/get-autoru-vos/209286...,85452,2014.0,"{'code': 'XC90', 'name': 'XC90', 'ru_name': 'X...",XC90,2.0d AT (235 л.с.) 4WD,5.0,1604647120000,RUR,2016,1101191623-464cf4be,"{'id': '20228874', 'name': 'II', 'ru_name': '2...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2799000
1,1,1,седан,VOLVO,https://auto.ru/cars/used/sale/volvo/s80/11013...,чёрный,{'id': '0'},При покупке автомобиля до 15 ноября \n► Выгод...,2.5 LTR,249 N12,Москва,"{'cruise-control': True, 'asr': True, 'tinted-...",бензин,//avatars.mds.yandex.net/get-autoru-vos/216441...,116410,2009.0,"{'code': 'S80', 'name': 'S80', 'ru_name': 'S80...",S80,2.5 AT (249 л.с.),4.0,1604659835000,RUR,2013,1101345022-3e0e4f8c,"{'id': '7974165', 'name': 'II Рестайлинг', 'ru...",SEDAN AUTOMATIC 2.5,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1277000
2,2,2,внедорожник 5 дв.,VOLVO,https://auto.ru/cars/new/sale/volvo/xc90/11009...,серый,"{'id': '22148941', 'name': 'Momentum', 'availa...",Официальный дилер Volvo в Москве и Московской ...,2.0 LTR,249 N12,Москва,"{'multi-wheel': True, 'heated-wash-system': Tr...",бензин,//avatars.mds.yandex.net/get-autoru-vos/216468...,0,2019.0,"{'code': 'XC90', 'name': 'XC90', 'ru_name': 'X...",XC90,2.0 AT (249 л.с.) 4WD,5.0,1604474020000,RUR,2020,1100911068-4dd8f660,"{'id': '21552979', 'name': 'II Рестайлинг', 'r...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,EUROPEAN,NaN,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,4861200
3,3,3,внедорожник 5 дв.,VOLVO,https://auto.ru/cars/used/sale/volvo/xc60/1100...,чёрный,{'id': '0'},Автодилер года 2020\n\nПобедитель в номинации ...,2.4 LTR,190 N12,Москва,"{'cruise-control': True, 'engine-proof': True,...",дизель,//avatars.mds.yandex.net/get-autoru-vos/209157...,83657,2013.0,"{'code': 'XC60', 'name': 'XC60', 'ru_name': 'X...",XC60,2.4d AT (190 л.с.) 4WD,5.0,1604656488000,RUR,2016,1100336634-a6643416,"{'id': '20012324', 'name': 'I Рестайлинг', 'ru...",ALLROAD_5_DOORS AUTOMATIC 2.4,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1869000
4,4,4,седан,VOLVO,https://auto.ru/cars/used/sale/volvo/s60/11014...,чёрный,{'id': '0'},Объявление №690\n\nАвтомобиль в отличном состо...,2.4 LTR,170 N12,Москва,"{'engine-proof': True, 'cruise-control': True,...",бензин,//avatars.mds.yandex.net/get-autoru-vos/210185...,118000,2004.0,"{'code': 'S60', 'name': 'S60', 'ru_name': 'S60...",S60,2.4 AT (170 л.с.),4.0,1604638764000,RUR,2006,1101497967-8c590a51,"{'id': '2309989', 'name': 'I Рестайлинг', 'ru_...",SEDAN AUTOMATIC 2.4,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,369000


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36210 entries, 0 to 36209
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            36210 non-null  int64  
 1   index                 36210 non-null  int64  
 2   bodyType              36210 non-null  object 
 3   brand                 36210 non-null  object 
 4   car_url               36210 non-null  object 
 5   color                 36210 non-null  object 
 6   complectation_dict    36210 non-null  object 
 7   description           35210 non-null  object 
 8   engineDisplacement    36210 non-null  object 
 9   enginePower           36210 non-null  object 
 10  location              36210 non-null  object 
 11  equipment_dict        36210 non-null  object 
 12  fuelType              36210 non-null  object 
 13  image                 36210 non-null  object 
 14  mileage               36210 non-null  int64  
 15  modelDate          

In [12]:
test.head(5)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,RUB,2014,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,RUB,2017,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,88000,2013,"{""code"":""SUPERB"",""name"":""Superb"",""ru_name"":""Су...",SUPERB,DSG 1.8 AMT (152 л.с.),5,1603226280,RUB,2014,1100658222,"{""id"":""20026323"",""nameplate"":""DSG"",""displaceme...",LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,95000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 AT (110 л.с.),5,1603226284,RUB,2014,1100937408,"{""id"":""20105521"",""displacement"":1598,""engine_t...",LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,58536,2008,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.8 AT (152 л.с.),5,1603226288,RUB,2012,1101037972,"{""id"":""4561004"",""displacement"":1798,""engine_ty...",LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

## Data Preprocessing

In [14]:
# ... 

In [15]:
# для baseline просто возьму пару схожих признаков без полной обработки
columns = ['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage']
train.dropna(subset=['productionDate','mileage', 'bodyType'], inplace=True)
train.dropna(subset=['price'], inplace=True)

In [16]:
col = 'bodyType'
new_col = col + '_c'

train[new_col] = train[col].apply(cm.clear_body_type)
test[new_col] = test[col].apply(cm.clear_body_type)

# train[col + '_c'].value_counts(), test[col + '_c'].value_counts()
columns = [new_col, 'brand', 'productionDate', 'engineDisplacement', 'mileage']

In [17]:
col = 'brand'

train[col] = train[col].apply(lambda s: s.strip())
test[col] = test[col].apply(lambda s: s.strip())

# train[col].value_counts(), test[col].value_counts()

In [18]:
# col = 'productionDate'

# train[col] = train[col].apply(lambda x: 2020 - x)
# test[col] = test[col].apply(lambda x: 2020 - x)

# train[col].value_counts(), test[col].value_counts()

In [19]:
col = 'engineDisplacement'

train[col] = train[col].apply(cm.clear_engine_disp).astype(np.float32)
test[col] = test[col].apply(cm.clear_engine_disp).astype(np.float32)

# train[col].value_counts(), test[col].value_counts()

In [20]:
# col = 'mileage'
# train[col].value_counts(), test[col].value_counts()

In [21]:
col = 'enginePower'
# train[col] = train[col].astype(dtype=np.float32)

train[col] = train[col].apply(cm.clear_engine_power).astype(np.float32)
test[col] = test[col].apply(cm.clear_engine_power).astype(np.float32)

columns.append(col)

In [22]:
col = 'Привод'

train[col] = train[col].apply(cm.clear_car_drives)
test[col] = test[col].apply(cm.clear_car_drives)

columns.append(col)

In [23]:
col = 'modelDate'

train[col] = train[col].astype(dtype=np.uint32)
test[col] = test[col].astype(dtype=np.uint32)

columns.append(col)

In [24]:
# col = 'Руль'

# train[col] = train[col].str.lower()
# train[col] = train[col].str.strip()

# test[col] = test[col].str.strip()
# test[col] = test[col].map({'Левый':'left', 'Правый':'right'})

# columns.append(col)

In [25]:
# col = 'ПТС'

# train[col] = train[col].str.lower()
# train[col] = train[col].str.strip()
# train[col] = train[col].map({'оригинал':'original', 'дубликат':'duplicate'})
# train[col] = train[col].fillna('other')


# test[col] = test[col].str.lower()
# test[col] = test[col].str.strip()
# test[col] = test[col].map({'оригинал':'original', 'дубликат':'duplicate'})
# test[col] = test[col].fillna('other')

# columns.append(col)

In [26]:
# col = 'numberOfDoors'

# train[col] = train[col].astype(dtype=np.uint32)
# test[col] = test[col].astype(dtype=np.uint32)

# columns.append(col)

In [27]:
train.dropna(subset=columns, inplace=True)

In [28]:
df_train = train[columns]
df_test = test[columns]

In [29]:
y = train['price']

In [30]:
# print(abc)

## Label Encoding

In [31]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
for colum in ['bodyType_c', 'brand', 'Привод']:
    data[colum] = data[colum].astype('category').cat.codes

In [33]:
data

,bodyType_c,brand,productionDate,engineDisplacement,mileage,enginePower,Привод,modelDate,sample
0,7,8,2014,1.2,74000,105.0,1,2013,0
1,7,8,2017,1.6,60563,110.0,1,2017,0
2,7,8,2014,1.8,88000,152.0,1,2013,0
3,7,8,2014,1.6,95000,110.0,1,2013,0
4,7,8,2012,1.8,58536,152.0,1,2008,0
...,...,...,...,...,...,...,...,...,...
70891,7,0,2015,3.0,82000,245.0,2,2014,1
70892,14,0,2020,2.0,0,245.0,2,2018,1
70893,7,0,2011,3.0,71000,300.0,2,2010,1
70894,17,0,2013,2.0,197000,211.0,2,2011,1


In [34]:
cat_features_ids = np.array([0, 1, 6])

In [35]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

## Train Split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [37]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [38]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

Точность наивной модели по метрике MAPE: 109.97%


# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

In [39]:
# model_simple = CatBoostRegressor(iterations = ITERATIONS,
#                           # learning_rate = LR,
#                           # depth = 10,
#                           random_seed = RANDOM_SEED,
#                           eval_metric='MAPE',
#                           custom_metric=['R2', 'MAE']
#                          )
                                 
# grid = {'depth': [4, 6, 10, None],
#         'learning_rate': [LR / 10, LR, LR * 3]}
                                 
# grid_search_result = model_simple.grid_search(grid,
#                                               X=X_train,
#                                               y=y_train,
#                                               # cat_features=cat_features_ids,
#                                               plot=False)
                                 
# # # # display(grid_search_result)

In [40]:
# display(grid_search_result)
# # 'params': {'depth': 10, 'learning_rate': 0.1}

In [41]:
# print(abc)

## Fit

In [42]:
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          depth = 10,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
model.fit(X_train, y_train,
         cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.9777415	test: 2.0169083	best: 2.0169083 (0)	total: 132ms	remaining: 10m 59s
100:	learn: 0.1907889	test: 0.1959445	best: 0.1959445 (100)	total: 3.72s	remaining: 3m
200:	learn: 0.1663783	test: 0.1727673	best: 0.1727622 (199)	total: 7.81s	remaining: 3m 6s
300:	learn: 0.1493621	test: 0.1578804	best: 0.1578804 (300)	total: 12s	remaining: 3m 7s
400:	learn: 0.1396055	test: 0.1503646	best: 0.1503646 (400)	total: 16.2s	remaining: 3m 6s
500:	learn: 0.1329574	test: 0.1461663	best: 0.1461663 (500)	total: 20.6s	remaining: 3m 4s
600:	learn: 0.1291076	test: 0.1443100	best: 0.1443100 (600)	total: 24.8s	remaining: 3m 1s
700:	learn: 0.1241555	test: 0.1419949	best: 0.1419784 (698)	total: 29.6s	remaining: 3m 1s
800:	learn: 0.1208758	test: 0.1406598	best: 0.1406395 (798)	total: 34.5s	remaining: 3m 1s
900:	learn: 0.1182838	test: 0.1398160	best: 0.1398160 (900)	total: 39.3s	remaining: 2m 58s
1000:	learn: 0.1157071	test: 0.1386835	best: 0.1386835 (1000)	total: 43.9s	remaining: 2m 55s
1100:	learn: 

In [43]:
# display(model.get_param('depth'))

In [44]:
model.save_model('catboost_single_model_baseline.model')

In [45]:
predict = model.predict(X_test)

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 13.50%


Точность модели по метрике MAPE: 17.11%

Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 17%!

# Submission

In [46]:
predict_submission = model.predict(X_sub)
predict_submission

array([ 631390.89503703,  891689.30211617,  876931.35028942, ...,
        276070.54318521, 1183657.55293881, 1075194.76500687])

In [47]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)

,sell_id,price
0,1100575026,6.313909e+05
1,1100549428,8.916893e+05
2,1100658222,8.769314e+05
3,1100937408,7.007584e+05
4,1101037972,8.314178e+05
5,1100912634,7.531567e+05
6,1101228730,6.821085e+05
7,1100165896,4.610507e+05
8,1100768262,2.140207e+06
9,1101218501,8.108782e+05


В итоге получили **MAPE 30%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing